In [3]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/kir/KIR_development/bin/imports.ipynb")
execute_notebook("/cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/alternate/sample_mapping.ipynb")

Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


###Gathering data from non-cancer, dbgap populations

In [4]:
len(population_dictionary)

10

In [5]:
population_dictionary.keys()

['2017_dbGap_phs000254.v2.p1_NHLBILungCF',
 '2017_dbGap_phs000632.v1.p1_NHLBIFamHC',
 '2017_dbGap_phs000362.v1.p1_NHLBIFamFAF',
 '2017_dbGap_phs000209.v13.p3_MESA',
 '2017_dbGap_phs000290.v1.p1_NHLBILungPAH',
 '2017_dbGap_phs000179.v5.p2_GenEpidCOPD',
 '2017_dbGap_phs000631.v1.p1_ARDSnet',
 '2017_dbGap_phs000291.v2.p1_NHLBILungCOPD',
 '2017_dbGap_phs000422.v1.p1_NHLBILungA',
 '2017_dbGap_phs000518.v1.p1_NHLBIFamIB']

In [6]:
all_sra_files, all_fastq_files, all_out_dirs = [], [], []
populations, samples = [], []
for i, population in enumerate(population_dictionary.keys()):
    # adding an s to sample_list
    
    sra_files = ['{0}{1}/sra/{2}'.format(base, population, sample) for sample in population_dictionary[population]]
    fastq_files = ['{0}{1}/sra/{2}.fastq'.format(base, population, sample.split('.')[0]) for sample in population_dictionary[population]]
    out_dirs = ['c{1}/{2}/'.format(base, population, sample.split('.')[0]) for sample in population_dictionary[population]]
    
    #all_sra_files.extend(sra_files)
    #all_fastq_files.extend(fastq_files)
    #all_out_dirs.extend(out_dirs)
    populations.extend([population for sample in population_dictionary[population]])
    samples.extend([sample.split('.')[0] for sample in population_dictionary[population]])

In [7]:
print populations[0]
print samples[0]
print len(populations), len(samples)

2017_dbGap_phs000254.v2.p1_NHLBILungCF
SRR201049
1758 1758


In [6]:
samples[:10]

['SRR201049',
 'SRR201052',
 'SRR201055',
 'SRR201058',
 'SRR201061',
 'SRR201063',
 'SRR201067',
 'SRR201071',
 'SRR201073',
 'SRR201078']

In [8]:
def create_cluster_script_type(populations, samples):

    new_script_file = '/cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/alternate/alt_to_fastq.sh'

    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        #out_file.write("#$ -t 1-{0}\n".format(len(sra_files)))
        out_file.write("#$ -t 1-1\n".format(len(samples)))
        out_file.write("#$ -l h_vmem=2G\n")
        out_file.write("#$ -tc 50\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set populations=({0})\n".format(" ".join(populations)))
        out_file.write("set samples=({0})\n".format(" ".join(samples)))
        out_file.write("\n")

        out_file.write("set population=$populations[$SGE_TASK_ID]\n")
        out_file.write("set sample=$samples[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        
        # Make new directories
        out_file.write("mkdir /nrnb/users/ramarty/alternate_pops/$population\n")
        out_file.write("mkdir /nrnb/users/ramarty/alternate_pops/$population/$sample\n")
        # others??
        
        # Move to working directory
        out_file.write("cd /nrnb/data/controlled/2017_dbGap_phs000179.v5.p2_GenEpidCOPD\n")
        
        # Get fastq from SRA and move to respective folder
        out_file.write("echo /cellar/users/ramarty/programs/sratoolkit.2.8.2-1-ubuntu64/bin/fastq-dump /nrnb/data/controlled/$population/sra/$sample\n")
        out_file.write("/cellar/users/ramarty/programs/sratoolkit.2.8.2-1-ubuntu64/bin/fastq-dump --split-files /nrnb/data/controlled/$population/sra/$sample.sra\n")
        #out_file.write("/cellar/users/ramarty/programs/sratoolkit.2.8.2-1-ubuntu64/bin/fastq-dump /nrnb/data/controlled/$population/sra/$sample.sra\n")
        out_file.write("mv /nrnb/data/controlled/2017_dbGap_phs000179.v5.p2_GenEpidCOPD/$sample* /nrnb/data/controlled/$population/.\n")
        out_file.write("echo Fastqs extracted.\n")
        out_file.write("date\n")
        
        # Remove duplicates from fastq file - picard?? 
        
        # concat into a single fastq file
        #out_file.write("cat /nrnb/data/controlled/$population/$sample\_1.fastq /nrnb/data/controlled/$population/$sample\_2.fastq > /nrnb/data/controlled/$population/$sample.fastq\n")
        out_file.write("echo Fastqs merged.\n")
        out_file.write("date\n")

In [9]:
create_cluster_script_type(populations, samples)

In [11]:
def create_cluster_script_gather(populations, samples):

    new_script_file = '/cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/alternate/alt_data_pull.sh'

    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        #out_file.write("#$ -t 1-{0}\n".format(len(sra_files)))
        out_file.write("#$ -t 1-500\n".format(len(samples)))
        out_file.write("#$ -l h_vmem=30G\n") ## what does the sizing need to be?? 
        out_file.write("#$ -tc 25\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set populations=({0})\n".format(" ".join(populations)))
        out_file.write("set samples=({0})\n".format(" ".join(samples)))
        out_file.write("\n")

        out_file.write("set population=$populations[$SGE_TASK_ID]\n")
        out_file.write("set sample=$samples[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        
        # Make new directories
        out_file.write("mkdir /nrnb/users/ramarty/alternate_pops/$population\n")
        out_file.write("mkdir /nrnb/users/ramarty/alternate_pops/$population/$sample\n")
        out_file.write("mkdir /nrnb/users/ramarty/alternate_pops/$population/$sample/features\n")
        out_file.write("mkdir /nrnb/users/ramarty/alternate_pops/$population/$sample/hlaHD\n")
        
        # Map to kir reference, strip to paired fastq, combine to single
        out_file.write("python /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/map_to_reference.py /nrnb/data/controlled/$population/$sample.fastq /nrnb/data/controlled/$population/$sample\_kir.bam /cellar/users/ramarty/Data/kir/ref/all_alleles cellar\n")
        out_file.write("echo Mapped to KIR.\n")
        out_file.write("python /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/convert_to_fastq.py /nrnb/data/controlled/$population/$sample\_kir.bam /nrnb/data/controlled/$population/$sample\_kir.fastq\n")
        out_file.write("echo Stripped reads.\n")   
        
        # Gather components from KIR and whole exome
        out_file.write("python /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/component_collection.py /nrnb/data/controlled/$population/$sample\_kir.fastq /nrnb/users/ramarty/alternate_pops/$population/$sample/features/ /cellar/users/ramarty/Data/kir/kmers/kmer_groups/component_and_four_mers.txt components_four_kir\n")
        out_file.write("echo Components gathered - KIR.\n")
        out_file.write("python /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/component_collection.py /nrnb/data/controlled/$population/$sample.fastq /nrnb/users/ramarty/alternate_pops/$population/$sample/features/ /cellar/users/ramarty/Data/kir/kmers/kmer_groups/component_and_four_mers.txt components_four\n")
        out_file.write("echo Components gathered - whole exome.\n")
        
        # Run PING
        out_file.write("cd /nrnb/users/ramarty/programs/PING\n") 
        out_file.write("Rscript --vanilla /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/runPING_extractor.R /nrnb/data/controlled/$population/ $sample\_1.fastq $sample\_2.fastq /nrnb/users/ramarty/alternate_pops/$population/$sample/PING_sequences/ 4\n")
        out_file.write("echo PING step 1 completed.\n")
        out_file.write("Rscript --vanilla /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/runPING_gc_caller.R /nrnb/users/ramarty/alternate_pops/$population/$sample/PING_sequences/ /nrnb/users/ramarty/alternate_pops/$population/$sample/PING/ 40000\n")
        out_file.write("echo PING step 2 completed.\n")
        
        # Run HLA-HD
        out_file.write("hlahd.sh -t 8 -m 70 -f ~/programs/hlahd.1.0.0/freq_data/ /nrnb/data/controlled/$population/$sample\_1.fastq /nrnb/data/controlled/$population/$sample\_2.fastq ~/programs/hlahd.1.0.0/HLA_gene.split.txt ~/programs/hlahd.1.0.0/dictionary/ $sample /nrnb/users/ramarty/alternate_pops/$population/$sample/hlaHD\n")
        out_file.write("echo HLA-HD completed.\n")
        
        # Remove fastq files
        out_file.write("rm /nrnb/data/controlled/$population/$sample*.fastq\n")
        out_file.write("rm /nrnb/data/controlled/$population/$sample*.bam\n")

        out_file.write("date\n")

In [12]:
create_cluster_script_gather(populations, samples)

In [ ]:
'''
# PING error message
Warning messages:
1: In file.remove(paste0(sequence, ".temp")) :
  cannot remove file '.temp', reason 'No such file or directory'
2: In file.remove(paste0("dump.me")) :
  cannot remove file 'dump.me', reason 'No such file or directory'
/cellar/users/ramarty/anaconda/lib/R/bin/R: 12: [: Linux: unexpected operator
Error in build_mira_counts(sequence_list) : 
  Program stopped, please check MIRA_log.txt for more information
Calls: ping_gc_caller -> build_mira_counts
Execution halted
'''